In [1]:
import subprocess 
import json
import time, os
import struct, socket
import numpy as np
import xml.etree.ElementTree as ET
import sys
import linecache, bisect

#**Variables declaration**

In [2]:
#variables
iploc = '/home/hadoop/ipython/iploc/iploc.csv'

#----------------- GTI Integration------------------------#
rest_client_path = '/home/hadoop/refclient/restclient'
gti_server = 'preprod.rest.gti.mcafee.com'
gti_user= 'int-dux-dev'
gti_password = '0d52ar50D{6cszTM!q<8V<Iw4GC52/'

gti_command = (rest_client_path + ' -s '+ gti_server 
               + ' -q \'{"ci":{"cliid":"87d8d1082c2f2f821f438b2359b7a5b4", "prn":"Duxbury", "sdkv":"1.0", "pv":"1.0.0", "pev":1, "rid":1, "affid":"0"},"q":[{"op":"ip","ip":"###IP###"}]}\''
               + ' -i '+ gti_user + ' -p \'' + gti_password + '\' -t')

#------------------ Norse---------------------------------#
norse_api_key = ''
norse_url = 'http://us.api.ipviking.com/api/'
norse_command = 'curl -d apikey={0} -d method=ipview -d ip={1} {2}'

#------- LDA Scores Files----------------------------------------#
spath = '/home/duxbury/ipython/user/20150418/'
lda_file = spath + 'lda_scores.csv'
lda_bu = spath + 'lda_scores_bu.csv'
lda_temp = spath + 'lda_scores_tmp.csv'
#!cp $lda_file $lda_bu
!cp $lda_file $lda_temp


print 'loading context....'
iplist = np.loadtxt(iploc,dtype=np.uint32,delimiter=',',usecols={0},
                    converters={0: lambda s: np.uint32(s.replace('"',''))})


loading context....


# Functions Definition

In [3]:
def ip_to_int(ip):
    o = map(int, ip.split('.'))
    res = (16777216 * o[0]) + (65536 * o[1]) + (256 * o[2]) + o[3]
    return res
def check_if_ip_is_internal(ip, ranges):
    for r in ranges:
        if(ip_to_int(ip) >= r[0] and ip_to_int(ip) <= r[1]):
            return 1
    return 0

def get_geo_ip(ip):
    result = linecache.getline(iploc, bisect.bisect(iplist,ip_to_int(ip))).replace('\n','')
    return result

def get_gti_rep(ip):
    reputation = ''
    try:
        response_json = subprocess.check_output(gti_command.replace('###IP###', ip), shell=True)
        result_dict = json.loads(response_json[0:len(response_json) - 1])
        reputation = result_dict['a'][0]['rep']
    except:
        reputation = 'error calling GTI'
    return reputation

#Currently it only returns the risk name... we can expand it to return a hash of interesting values
def get_norse_rep(ip):
    reputation = ''
    try:
        response = subprocess.check_output(norse_command.format(norse_api_key, ip, norse_url), shell=True)
        # the executed curl command returns an XML as response 
        root_element = ET.fromstring(response)
        reputation = root_element.find("./response/risk_name").text
    except:
        reputation = 'error calling norse'
    
    return reputation


#Load Internal IP Context

In [4]:
#Adds the network internal IP ranges to a matrix
ranges_file_path = 'InternalIP_Ranges.csv'
ranges = []

f = open(ranges_file_path, 'r')

for line in f:
    line_parts = line.split(',');
    ranges.append([ip_to_int(line_parts[0]), ip_to_int(line_parts[1])])

print ranges
f.close()

[[3328005120, 3328006143], [3232301312, 3232302079], [2998054912, 2998055679], [2411322880, 2411323135], [3224646400, 3224648959], [3224838144, 3224889855], [3227956736, 3227964415], [3234234368, 3234247167], [3333373952, 3333389311], [3430987520, 3430987775], [2260715520, 2260729855], [2161573888, 2161639423], [2229862400, 2229927935], [2256928768, 2256994303], [2301493248, 2301558783], [2459435008, 2459500543], [2479882240, 2479947775]]


#Read lda csv file:
Add Network context, Reputation Services and Geo IP location

**Note:** You can comment one or more code sections in order to add just a part of the new information. Code sections look like this:

\#--------------- Section ----------------#

In [5]:
#import csv
ip_dict = {}
new_content = ''
with open(lda_bu, 'r+') as lda_f:
    for line in lda_f:    
        if(new_content==""):
              
            new_content += line[:-1] 
            
            #--------------------- Add Context and Geo location Headers----------------------#            
            new_content += ',srcIpInternal,destIpInternal,srcGeo,dstGeo,srcDomain,dstDomain'
            
            #--------------------- Add GTI Reputation Headers----------------------#
            new_content += ',gtiSrcRep,gtiDstRep'
            
            #--------------------- Add Norse reputation Headers----------------------#
            #new_content += ',norseSrcRep,norseDstRep'
            
            
            new_content += '\n'
            
        else:
            line_parts = line.split(',')
            src_ip = line_parts[2]
            dst_ip = line_parts[3]
            is_src_ip_internal = check_if_ip_is_internal(src_ip, ranges) # index 2 is the srcIP
            is_dest_ip_internal = check_if_ip_is_internal(dst_ip, ranges) # index 3 is the destip           
            
            #Add current line content... chop '\n' character
            new_content += line[:-1] 
            
            #--------------------- Add Context ----------------------#          
            #new_content += ',' + str(is_src_ip_internal) + ',' + str(is_dest_ip_internal) 
            
            src_geo = ";".join(get_geo_ip(src_ip).replace('"','').split(',')[4:6])+ " " + ";".join(get_geo_ip(src_ip).replace('"','').split(',')[8:9])
            dst_geo = ";".join(get_geo_ip(dst_ip).replace('"','').split(',')[4:6]) + " " + ";".join(get_geo_ip(dst_ip).replace('"','').split(',')[8:9])            
            src_domain = get_geo_ip(src_ip).replace('"','').split(',')[9:10][0]
            dst_domain = get_geo_ip(dst_ip).replace('"','').split(',')[9:10][0]            
            #check if src IP is not in dictionary and add it if true
            if src_ip not in ip_dict:                                
                if is_src_ip_internal:
                    ip_dict[src_ip] = { 'geo': src_geo
                                       ,'domain': src_domain 
                                       ,'gti_rep': ''
                                       #,'norse_rep': ''
                                      }
                else:                   
                    ip_dict[src_ip] = { 'geo': src_geo
                                       ,'domain': src_domain
                                       ,'gti_rep': get_gti_rep(src_ip)
                                       #,'norse_rep': get_norse_rep(src_ip) 
                                      }                   
            
            if dst_ip not in ip_dict:
                if is_dest_ip_internal:                    
                    ip_dict[dst_ip] = { 'geo': dst_geo
                                       ,'domain': dst_domain
                                       ,'gti_rep': ''
                                       #,'norse_rep': ''
                                      }
                else:                   
                    ip_dict[dst_ip] = { 'geo': dst_geo
                                       ,'domain': dst_domain
                                       ,'gti_rep': get_gti_rep(dst_ip)
                                       #,'norse_rep':get_norse_rep(dst_ip) 
                                      }
                   
            
            #print src_ip
            #print get_geo_ip(src_ip)
            
            #------Add Geo Location-------------#
            new_content = new_content + "," + ip_dict[src_ip]['geo'] + ',' +ip_dict[dst_ip]['geo']
            
            #------Add Domain-------------------#
            new_content = new_content + "," + ip_dict[src_ip]['domain'] + ',' +ip_dict[dst_ip]['domain']
            
            #--------Add GTI Reputation---------#
            #print str(ip_dict[src_ip]['gti_rep'])
            #print str(ip_dict[dst_ip]['gti_rep'])
            new_content += "," +  str(ip_dict[src_ip]['gti_rep']) + ',' +  str(ip_dict[dst_ip]['gti_rep'])
            
            #-------Add Norse Rep --------------#
            #new_content += "," +  ip_dict[src_ip]['norse_rep'] + ',' +  ip_dict[dst_ip]['norse_rep']
            
            
            new_content += "\n"
            
print new_content
        
    

sev,tstart,srcIP,dstIP,sport,dport,proto,ipkt,ibyt,lda_score,srcIpInternal,destIpInternal,srcGeo,dstGeo,srcDomain,dstDomain,gtiSrcRep,gtiDstRep,norseSrcRep,norseDstRep
0,2015-04-18 07:44:17,109.107.35.58,192.55.54.42,0,768,ICMP,1,80,2.93445652899e-09,0,1,England;Leeds Brightbox Systems Ltd,California;Folsom Intel Corporation,brightbox.co.uk,intel.com,22,,Low,
0,2015-04-18 18:34:38,74.208.82.239,134.134.137.75,21,63712,TCP,8,410,2.9425881171e-09,0,1,Missouri;Kansas City 1&1 Internet Inc.,Oregon;Hillsboro Intel Corporation,oneandone.net,intel.com,22,,Low,
0,2015-04-18 11:37:44,59.151.12.228,134.134.137.75,81,55868,TCP,9,1938,2.9425881171e-09,0,1,Beijing;Beijing 21Vianet (China) Inc.,Oregon;Hillsboro Intel Corporation,21vianet.com,intel.com,22,,Low,
0,2015-04-18 18:06:41,23.62.236.16,192.55.55.41,443,64357,TCP,71637,100235859,2.97576187409e-09,0,1,California;San Jose Akamai Technologies Inc.,California;Folsom Intel Corporation,akamai.com,intel.com,22,,Low,
0,2015-04-18 22:57:45,123.151.42

In [6]:
f = open(lda_bu, 'w')
f.write(new_content)
f.close()

f2 = open(lda_file, 'w')
f2.write(new_content)
f2.close()
